In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# import jtplot submodule from jupyterthemes
from jupyterthemes import jtplot

# currently installed theme will be used to
# set plot style if no arguments provided
jtplot.style()

from preprocessing import *

data_path = Path('tbmData/data')
fn_txt = sorted(data_path.glob('*.txt')); print(fn_txt[:3])
fn_cycles = Path('tbmData/cycles1')

num_cycles = 3481
valid_ratio = 0.2
train_ratio = 1 - valid_ratio
valid_idx = range(int(num_cycles * train_ratio), num_cycles)

mulr = 100
bs = int(num_cycles * train_ratio) 

cont_names, dep_var = ['推进速度', '主驱动1#电机扭矩', '刀盘扭矩', '刀盘转速', '主液压油箱温度'], '桩号'

In [ ]:
# step = 201
# for i in range(0, 802, step):
#     print(f'reading [{i}, {i+step})')
#     formatter = DataFormatter(fn_txt[i:i+step])
#     print('cleaning up')
#     formatter.cleanup()
#     print('cleanup done')
#     formatter.cycles1(fn='tmp/cycles/cycle', first_idx=i)
#     print('write done')

In [ ]:
# cycles = [feather.read_dataframe(str(o)) for o in sorted(fn_cycles.glob('cycle*'))[:num_cycles]]

# cont_names = ['推进速度', '主驱动1#电机扭矩', '刀盘扭矩', '刀盘转速', '主液压油箱温度']
# dep_var = '桩号'

# beginning_idx = [beginning_index(c) for c in cycles]
# cont = [o[cont_names].iloc[i:i+30] for i, o in zip(beginning_idx, cycles)]

In [ ]:
with concurrent.futures.ThreadPoolExecutor() as e:
    def f(o):
        return feather.read_dataframe(str(o))
    cycles = list(tqdm_notebook(e.map(f, sorted(fn_cycles.glob('cycle*'))[:num_cycles]), desc='read_dataframe', total=num_cycles))
    idx = tqdm_notebook(e.map(beginning_index, [o.iloc[:500] for o in cycles]), desc='beginning', total=num_cycles)
    cont = [o[cont_names].iloc[i:i+30] for i, o in zip(idx, cycles)]

In [ ]:
%%time
cyc_cont = pd.DataFrame([o.values.flatten() for o in cont])
dep = pd.DataFrame({'桩号': [o['桩号'].iloc[0] for o in cycles]})
index = pd.Series(cyc_cont.index)
cyc_cont = pd.concat([index, cyc_cont, dep], axis=1, copy=False)
cyc_cont.columns = [str(o) for o in range(cyc_cont.shape[1]-1)] + [dep_var]
print(cyc_cont.shape)
cyc_cont.head()

In [ ]:
cyc_cont.to_feather('tmp/cyc_cont_all')

In [ ]:
cyc_cont = feather.read_dataframe('tmp/cyc_cont_all')

In [ ]:
stacked = pd.concat([cyc_cont]*mulr)
print(stacked.shape)
procs = Normalize
databunch = TabularDataBunch.from_df('tmp', stacked, dep_var, valid_idx=valid_idx, bs=bs, procs=procs)
learner = tabular_learner(databunch, [600, 600, 600, 600, 100])
learner.callback_fns.append(ShowGraph)
learner.model

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(10, 5 * 1e-2)

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(5, 1e-2)
learner.save('fc3.1')

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(5, 1e-4)
learner.save('fc3.2')

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(5, 1e-5)
learner.save('fc3.3')

In [ ]:
x, y = next(iter(databunch.valid_dl))
y = to_np(y)
pred = to_np(learner.pred_batch(x)).squeeze()

In [ ]:
list(zip(pred, y, [(a-b)/(71476-51705)*100 for a,b in zip(pred, y)]))

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.fit_one_cycle(5, 1e-5/2)
learner.save('fc3.4')